**Reasoning**:
Install the gradio library using pip.



In [7]:
import fitz  # PyMuPDF
import re
import spacy
from spacy.matcher import Matcher

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# ========== PDF Text Extraction using fitz ==========
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# ========== Cleaning Broken Lines ==========
def clean_text(text):
    lines = text.split('\n')
    cleaned_lines = []
    i = 0
    while i < len(lines):
        if i < len(lines)-1 and lines[i].istitle() and lines[i+1].istitle():
            cleaned_lines.append(lines[i] + " " + lines[i+1])
            i += 2
        else:
            cleaned_lines.append(lines[i])
            i += 1
    return "\n".join(cleaned_lines)

# ========== Extract Name ==========
def extract_name(text):
    doc = nlp(text)
    matcher = Matcher(nlp.vocab)
    matcher.add("NAME", [[{'POS': 'PROPN'}, {'POS': 'PROPN'}]])
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text
    return None

# ========== Extract Email ==========
def extract_email(text):
    # Normalize whitespace and remove newline breaks that might split emails
    cleaned_text = re.sub(r'[\s\n]+', '', text)

    # Try a relaxed pattern to capture email-like strings
    pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"
    emails = re.findall(pattern, cleaned_text)
    return emails[0] if emails else None


# ========== Extract Phone ==========
def extract_phone(text):
    cleaned_text = ' '.join(text.split())  # Normalize whitespace
    phones = re.findall(r"(?:\+977[-.\s]?)?[9][6-9]\d{7,8}", cleaned_text)
    return phones[0] if phones else None

# ========== Extract Skills ==========
def extract_skills(text, skills_list):
    found = []
    for skill in skills_list:
        if re.search(rf"\b{re.escape(skill)}\b", text, re.IGNORECASE):
            found.append(skill)
    return found

# ========== Extract Education ==========
def extract_education(text):
    pattern = r"(?i)(Bachelors?|Bachelor’s?|Technical Education).+?(?:\n|$)"
    return [match.strip() for match in re.findall(pattern, text)]

# ========== Main Program ==========
def extract_resume_info(pdf_path):
    """
    Extracts resume information from a PDF file.

    Args:
        pdf_path: The path to the PDF file.

    Returns:
        A dictionary containing the extracted information or an error message.
    """
    try:
        raw_text = extract_text_from_pdf(pdf_path)
        text = clean_text(raw_text)

        # Define skill keywords - keeping this inside the function as it's specific to the extraction logic
        skills_list = [
            "React", "JavaScript", "Python", "Machine Learning", "AI", "NLP", "Laravel",
            "YOLOv8", "Frontend Development", "Backend Development", "IOT Development"
        ]

        name = extract_name(text)
        email = extract_email(text)
        phone = extract_phone(text)
        skills = extract_skills(text, skills_list)
        education = extract_education(text)

        extracted_info = {
            'Name': name,
            'Email': email,
            'Phone': phone,
            'Skills': skills,
            'Education': education
        }

        return extracted_info
    except Exception as e:
        return {"error": str(e)}

# Example usage (optional, for testing)
# resume_path = "Saugat-Dhungana-cv.pdf"
# extracted_data = extract_resume_info(resume_path)
# print(extracted_data)

In [8]:
import gradio as gr

# Define the Gradio interface
iface = gr.Interface(
    fn=extract_resume_info,
    inputs=gr.File(label="Upload PDF Resume"),
    outputs=gr.JSON(label="Extracted Resume Information"),
    title="Resume Information Extractor",
    description="Upload a PDF resume to extract key information like name, email, phone, skills, and education."
)

# To launch the interface, you would typically use:
# iface.launch()
# However, for this subtask, we only need to define the interface.

In [9]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e45ee90e74b85dc73.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   The `gradio` library was already installed in the environment before the process began.
*   A Python function `extract_resume_info` was successfully created to encapsulate the PDF text extraction and information parsing logic.
*   A Gradio interface was successfully defined using `gradio.Interface`, featuring a file upload input for PDF files and a JSON output to display extracted information.
*   The Gradio application was successfully launched and made publicly accessible via a URL, as the environment was detected as a hosted notebook.

### Insights or Next Steps

*   The created Gradio application provides a user-friendly interface for extracting structured data from PDF resumes, which can be valuable for recruitment or database population tasks.
*   Further development could involve improving the robustness of the extraction functions to handle a wider variety of resume formats and potentially adding more extraction fields like work experience or projects.
